# Evaluación de Fréchet Audio Distance (FAD)

Este notebook calcula la distancia Fréchet Audio Distance (FAD) entre diferentes conjuntos de muestras de audio utilizando embeddings CLAP. FAD es una métrica utilizada para evaluar la calidad y diversidad del audio generado comparando la distribución de embeddings entre muestras reales y generadas.

**Puntajes FAD más bajos indican mejor calidad** - la distribución del audio generado está más cerca de la distribución del audio de referencia.

## Configuración e Importaciones

Importar las bibliotecas necesarias para la extracción de embeddings de audio, cálculos estadísticos y configuración del proyecto.

In [1]:
import glob
from pathlib import Path
import numpy as np
from scipy import linalg
import logging

from models.clap_score import ClapModel
from config import setup_project_paths, load_config, PROJECT_ROOT

/Users/mario/PycharmProjects/audio_reprompt/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Cálculo de Distancia de Fréchet

La distancia de Fréchet mide la similitud entre dos distribuciones gaussianas multivariadas. Se calcula utilizando los vectores de media (μ) y las matrices de covarianza (Σ) de ambas distribuciones:

$$d^2 = ||\mu_1 - \mu_2||^2 + \text{Tr}(\Sigma_1 + \Sigma_2 - 2\sqrt{\Sigma_1 \Sigma_2})$$

Esta función maneja problemas de estabilidad numérica que pueden surgir durante el cálculo de la raíz cuadrada de matrices.

In [2]:
def calculate_frechet_distance(mu1, sigma1, mu2, sigma2, eps=1e-6):
    """Calcula la distancia de Fréchet entre dos distribuciones gaussianas."""
    mu1 = np.atleast_1d(mu1)
    mu2 = np.atleast_1d(mu2)
    sigma1 = np.atleast_2d(sigma1)
    sigma2 = np.atleast_2d(sigma2)

    diff = mu1 - mu2
    covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)

    if not np.isfinite(covmean).all():
        offset = np.eye(sigma1.shape[0]) * eps
        covmean = linalg.sqrtm((sigma1 + offset).dot(sigma2 + offset))

    if np.iscomplexobj(covmean):
        covmean = covmean.real

    tr_covmean = np.trace(covmean)

    return diff.dot(diff) + np.trace(sigma1) + np.trace(sigma2) - 2 * tr_covmean

## Cálculo de FAD usando Embeddings CLAP

Esta función extrae embeddings CLAP de archivos de audio y calcula el puntaje FAD entre dos conjuntos de muestras de audio.

In [3]:
def calculate_fad(path_audio_samples: str| list[str], path_generated_samples: str|list[str], taste_paths: bool = False):
    """Calcula el puntaje FAD usando embeddings CLAP. Menor es mejor."""

    if taste_paths:
        prompt_audios = path_audio_samples
        reprompt_audios = path_generated_samples
    else:
        prompt_audios = [str(path) for path in glob.glob(f"{path_audio_samples}/*.wav")]
        reprompt_audios = [str(path) for path in glob.glob(f"{path_generated_samples}/**/*.wav")]

    real_embeddings = model.embed_audio(prompt_audios).numpy()

    gen_embeddings = model.embed_audio(reprompt_audios).numpy()

    mu_real = np.mean(real_embeddings, axis=0)
    sigma_real = np.cov(real_embeddings, rowvar=False)

    mu_gen = np.mean(gen_embeddings, axis=0)
    sigma_gen = np.cov(gen_embeddings, rowvar=False)

    return calculate_frechet_distance(mu_real, sigma_real, mu_gen, sigma_gen)

## Comparación de FAD por Categoría de Sabor

Compara muestras de audio filtradas por categoría de sabor (dulce, amargo, salado, ácido). Esto permite evaluar qué tan bien el modelo genera audio para perfiles de sabor específicos.

In [4]:
def compare_audio_taste_samples(spanio_track_dir: str, reprompt_track_dir: str, taste_value: str):
    """Calcula FAD para una categoría de sabor específica."""
    prompt_audios = [str(path) for path in glob.glob(f"{spanio_track_dir}/{taste_value}_*.wav")]
    reprompt_audios = [str(path) for path in glob.glob(f"{reprompt_track_dir}/{taste_value}/*.wav")]

    return calculate_fad(prompt_audios, reprompt_audios, taste_paths=True)

## Configuración y Rutas

Cargar la configuración del proyecto y definir las rutas a los directorios de muestras de audio.

In [5]:
setup_project_paths()
config = load_config()
AUDIOS_PATH = (PROJECT_ROOT / config.data.tracks_data_path).parent

audios_without_reprompt = AUDIOS_PATH / "raw_prompts_audios"
audios_with_reprompt = AUDIOS_PATH / "reprompt_audio_taste"
audios_spanio = AUDIOS_PATH / "generated_base_music"

model = ClapModel(device="auto", enable_fusion=True)

/Users/mario/PycharmProjects/audio_reprompt/.venv/lib/python3.11/site-packages/numba/cpython/hashing.py:477: UserWarning: FNV hashing is not implemented in Numba. See PEP 456 https://www.python.org/dev/peps/pep-0456/ for rationale over not using FNV. Numba will continue to work, but hashes for built in types will be computed using siphash24. This will permit e.g. dictionaries to continue to behave as expected, however anything relying on the value of the hash opposed to hash as a derived property is likely to not work as expected.
  warnings.warn(msg)
/Users/mario/PycharmProjects/audio_reprompt/.venv/lib/python3.11/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3550.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of RobertaModel were not initialized f

Load our best checkpoint in the paper.
The checkpoint is already downloaded
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.patch_embed.mel_conv2d.weight 	 Loaded
audio_branch.patch_embed.mel_conv2d.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.3.weight 	 Loaded
audio_branc

## Resultados de Evaluación

### FAD General: Prompt vs Reprompt

Comparar el audio de prompts originales con el audio reprompted para medir el efecto del reprompting.

In [6]:
fad_prompt_vs_reprompt = calculate_fad(audios_without_reprompt, audios_with_reprompt)
print(f"FAD (Prompt vs Reprompt): {fad_prompt_vs_reprompt:.4f}")

/Users/mario/PycharmProjects/audio_reprompt/.venv/lib/python3.11/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


FAD (Prompt vs Reprompt): 0.1264


/var/folders/p3/7n0xc16x2cz3psh_2bhr4m8w0000gn/T/ipykernel_15559/2467182017.py:9: DeprecationWarning: The `disp` argument is deprecated and will be removed in SciPy 1.18.0.
  covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)


### FAD General: Spanio vs Reprompt

Comparar la música base generada (Spanio) con el audio reprompted.

In [7]:
fad_spanio_vs_reprompt = calculate_fad(audios_spanio, audios_with_reprompt)
print(f"FAD (Spanio vs Reprompt): {fad_spanio_vs_reprompt:.4f}")

FAD (Spanio vs Reprompt): 0.4397


/var/folders/p3/7n0xc16x2cz3psh_2bhr4m8w0000gn/T/ipykernel_15559/2467182017.py:9: DeprecationWarning: The `disp` argument is deprecated and will be removed in SciPy 1.18.0.
  covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)


### Puntajes FAD por Sabor

Calcular FAD para cada categoría de sabor para evaluar qué tan bien el modelo captura perfiles de sabor específicos.

In [8]:
taste_categories = ["sweet", "bitter", "salty", "sour"]

print("Puntajes FAD por Sabor (Spanio vs Reprompt):")

for taste in taste_categories:
    fad_score = compare_audio_taste_samples(audios_spanio, audios_with_reprompt, taste)
    print(f"FAD ({taste.capitalize()}): {fad_score:.4f}")

Puntajes FAD por Sabor (Spanio vs Reprompt):


/var/folders/p3/7n0xc16x2cz3psh_2bhr4m8w0000gn/T/ipykernel_15559/2467182017.py:9: DeprecationWarning: The `disp` argument is deprecated and will be removed in SciPy 1.18.0.
  covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)


FAD (Sweet): 0.5893
FAD (Bitter): 0.7034
FAD (Salty): 0.5969
FAD (Sour): 0.5381


## Resumen

Este notebook evalúa la calidad de generación de audio usando puntajes FAD:

- **Valores FAD más bajos** indican mejor alineación entre distribuciones
- **FAD general** proporciona una métrica de calidad general
- **FAD por sabor** muestra qué tan bien el modelo captura categorías individuales de sabor